In [1]:
# Cell 1: Import libraries
import os
import json
import re
import base64  # New import for base64 encoding
from faker import Faker
from datetime import datetime, timedelta
import random
import pandas as pd
import ollama
from pydantic import BaseModel, Field
from typing import List, Dict
from jinja2 import Environment, FileSystemLoader
import pdfkit


# Initialize Faker
fake = Faker()


In [2]:
# Cell 2: Directory setup and bank configuration
SAMPLE_LOGOS_DIR = "sample_logos"
SYNTHETIC_STAT_DIR = "new_statements"
TEMPLATES_DIR = "templates"

# Create directories if they don’t exist
for directory in [SAMPLE_LOGOS_DIR, SYNTHETIC_STAT_DIR, TEMPLATES_DIR]:
    os.makedirs(directory, exist_ok=True)

# Bank configuration: map bank names to logos, templates, and specific account names
BANK_CONFIG = {
    "chase": {
        "logo": "chase_bank_logo.png",
        "templates": ["chase_classic_style.html", "chase_variation_1.html", "chase_variation_2.html"],
        "account_types": {
            "personal": "Chase Total Checking",
            "business": "Chase Business Complete Checking"
        }
    },
    "citibank": {
        "logo": "citibank_logo.png",
        "templates": ["citibank_classic_template.html", "citibank_variation_1.html", "citibank_variation_2.html"], 
        "account_types": {
            "personal": "Citi Access Checking",
            "business": "CitiBusiness Checking"
        }
    },
    "wellsfargo": {
        "logo": "wellsfargo_logo.png",
        "templates": ["wells_fargo_classic.html", "wells_variation_1.html", "wells_variation_2.html"],  
        "account_types": {
            "personal": "Everyday Checking",
            "business": "Initiate Business Checking"
        }
    },
    "pnc": {
        "logo": "pnc_logo.png",
        "templates": ["pnc_classic.html"],
        "account_types": {
            "personal": "Standard Checking",
            "business": "Business Checking"
        }
    }
}

# Validate bank configuration
for bank, config in BANK_CONFIG.items():
    logo_path = os.path.join(SAMPLE_LOGOS_DIR, config["logo"])
    for template in config["templates"]:
        template_path = os.path.join(TEMPLATES_DIR, template)
        if not os.path.exists(template_path):
            raise FileNotFoundError(f"Template file not found for {bank}: {template_path}")
    if not os.path.exists(logo_path):
        raise FileNotFoundError(f"Logo file not found for {bank}: {logo_path}")

In [3]:
# Cell 3: Pydantic models
class FieldDefinition(BaseModel):
    name: str = Field(..., description="Field name (e.g., account_holder, account_number)")
    is_mutable: bool = Field(..., description="Whether the field is mutable")
    description: str = Field(..., description="Description of the field")

class StatementFields(BaseModel):
    fields: List[FieldDefinition] = Field(..., description="List of mutable and immutable fields")

class Transaction(BaseModel):
    description: str = Field(..., max_length=35, description="Transaction description")
    category: str
    amount: float
    account_type: str = Field(..., description="Type of account (business or personal)")

In [4]:
# Cell 4: Generate category lists
def generate_category_lists(account_type: str) -> tuple[List[str], List[str]]:
    account_context = "business" if account_type == "business" else "personal"
    prompt = f"""
    Generate two lists of bank transaction categories in JSON format for {account_context} bank statements.
    One list for reasons someone loses money (e.g., {'vendor payments, payroll' if account_type == 'business' else 'utilities, subscriptions'}) and one for reasons someone gains money (e.g., {'client invoices, refunds' if account_type == 'business' else 'deposits, refunds'}).
    Each list should have 5 unique categories, each 1-2 words, title case, no punctuation.
    Return:
    {{
      "loss_categories": ["Category One", "Category Two", ...],
      "gain_categories": ["Category One", "Category Two", ...]
    }}
    """
    try:
        response = ollama.generate(model="mistral:7b-instruct-v0.3-q4_0", prompt=prompt)
        category_data = json.loads(response['response'].strip())
        loss_categories = [cat for cat in category_data.get("loss_categories", []) if isinstance(cat, str) and 1 <= len(cat.split()) <= 2]
        gain_categories = [cat for cat in category_data.get("gain_categories", []) if isinstance(cat, str) and 1 <= len(cat.split()) <= 2]
        if len(loss_categories) < 5 or len(gain_categories) < 5:
            raise ValueError("Insufficient valid categories")
    except (json.JSONDecodeError, ValueError):
        if account_type == "business":
            loss_categories = ["Vendor Payment", "Payroll Expense", "Office Supplies", "Equipment Purchase", "Marketing Cost"]
            gain_categories = ["Client Invoice", "Refund Received", "Investment Income", "Grant Received", "Sales Revenue"]
        else:  # personal
            loss_categories = ["Utility Payment", "Subscription Fee", "Online Purchase", "Rent Payment", "Grocery Shopping"]
            gain_categories = ["Salary Deposit", "Tax Refund", "Gift Received", "Client Payment", "Cash Deposit"]
    return loss_categories, gain_categories

In [5]:
# Cell 5: Generate transaction description
def generate_transaction_description(amount: float, category: str, account_type: str) -> dict:
    account_context = "business" if account_type == "business" else "personal"
    prompt = f"""
    Generate a bank transaction description (3-5 words, max 45 characters) for a {account_context} bank transaction in the '{category}' category.
    Rules:
    - Use title case.
    - No punctuation.
    - No parentheses, dashes, or dollar signs.
    - No amounts or numbers as words.
    - Use simple phrases relevant to {account_context} accounts.
    - Examples: {'Office Supply Purchase' if account_type == 'business' else 'Grocery Store Purchase'}, {'Vendor Invoice Payment' if account_type == 'business' else 'Utility Bill Payment'}
    """
    try:
        response = ollama.generate(model="mistral:7b-instruct-v0.3-q4_0", prompt=prompt)
        description = response['response'].strip()[:25]
    except:
        description = f"{category} Transaction"
    description = description.replace("(", "").replace(")", "").replace(",", "").replace(":", "").replace("-", "").replace("$", "").replace(".", "")
    description = ' '.join(word.capitalize() for word in description.split())[:25]
    words = description.split()
    if len(words) < 3 or len(words) > 5:
        description = f"{category} Transaction"[:45]
    transaction = Transaction(description=description, category=category, amount=amount, account_type=account_type)
    return transaction.model_dump()

In [6]:
# Cell 6: Generate synthetic bank statement
def generate_bank_statement(num_transactions: int, account_holder: str, account_type: str) -> pd.DataFrame:
    if account_type not in ["business", "personal"]:
        raise ValueError("Account type must be 'business' or 'personal'")
    if not (3 <= num_transactions <= 25):  # Match app's validation
        raise ValueError("Number of transactions must be between 3 and 25")
    
    loss_categories, gain_categories = generate_category_lists(account_type)
    start_date = datetime.now() - timedelta(days=30)
    dates = [start_date + timedelta(days=random.randint(0, 30)) for _ in range(num_transactions)]
    
    # Ensure a mix of deposits and withdrawals
    transactions = []
    min_deposits = max(1, num_transactions // 3)  # Ensure at least 1/3 are deposits
    min_withdrawals = max(1, num_transactions // 3)  # Ensure at least 1/3 are withdrawals
    deposit_count = 0
    withdrawal_count = 0
    
    for _ in range(num_transactions):
        if deposit_count < min_deposits:
            is_gain = True
        elif withdrawal_count < min_withdrawals:
            is_gain = False
        else:
            is_gain = random.choice([True, False])
        
        category = random.choice(gain_categories if is_gain else loss_categories)
        amount = round(random.uniform(50, 1000), 2) if is_gain else round(random.uniform(-500, -10), 2)
        transaction = generate_transaction_description(amount, category, account_type)
        transactions.append(transaction)
        
        if is_gain:
            deposit_count += 1
        else:
            withdrawal_count += 1
    
    data = {
        "Date": [d.strftime("%m/%d") for d in dates],
        "Description": [t["description"] for t in transactions],
        "Category": [t["category"] for t in transactions],
        "Amount": [t["amount"] for t in transactions],
        "Type": [t["type"] for t in transactions],  # Added Type column
        "Balance": [0.0] * num_transactions,
        "Account Holder": [account_holder] * num_transactions,
        "Account Type": [account_type.capitalize()] * num_transactions,
        "Transaction ID": [(fake.bban()[:10] + str(i).zfill(4)) for i in range(num_transactions)]
    }
    df = pd.DataFrame(data)
    df = df.sort_values("Date")
    initial_balance = round(random.uniform(1000, 20000), 2)
    df["Balance"] = initial_balance + df["Amount"].cumsum()
    return df

In [7]:
# Cell 7: Identify mutable and immutable fields using Ollama
def identify_template_fields(bank: str, templates_dir: str = TEMPLATES_DIR) -> StatementFields:
    if bank not in BANK_CONFIG:
        raise ValueError(f"Unsupported bank: {bank}. Supported banks: {list(BANK_CONFIG.keys())}")
    
    template_path = os.path.join(templates_dir, BANK_CONFIG[bank]["templates"][0])
    if not os.path.exists(template_path):
        raise FileNotFoundError(f"Template file not found: {template_path}")
    
    with open(template_path, 'r', encoding='utf-8') as f:
        template_content = f.read()
    
    placeholders = re.findall(r'\{\{([^{}]+)\}\}', template_content)
    placeholders = [p.strip() for p in placeholders]
    
    prompt = f"""
    Given the following list of placeholders extracted from a {bank.capitalize()} bank statement HTML template, classify each as mutable (can be changed with synthetic data) or immutable (static, e.g., bank details or table headers). Return a JSON object with a list of fields, each containing the field name, whether it is mutable (true/false), and a brief description of what the field represents. Example:
    {{
        "fields": [
            {{"name": "account_holder", "is_mutable": true, "description": "Name of the account holder"}},
            {{"name": "bank_name", "is_mutable": false, "description": "Name of the bank"}}
        ]
    }}
    Placeholders:
    {', '.join(placeholders)}
    Rules:
    - Mutable fields include account holder, client number, date of birth, account number, IBAN, statement period, statement date, transactions, opening balance, total debit, total credit, total balance, and other customer-specific data.
    - Immutable fields include bank name, bank address, table headers, customer service information, and footnotes.
    - Ensure descriptions are relevant to {bank.capitalize()} bank statements.
    """
    try:
        response = ollama.generate(model="mistral:7b-instruct-v0.3-q4_0", prompt=prompt)
        fields_data = json.loads(response['response'].strip())
        statement_fields = StatementFields(**fields_data)
    except (json.JSONDecodeError, ValueError, ollama.RequestError) as e:
        print(f"Ollama failed for {bank}: {e}. Using fallback fields.")
        default_fields = [
            FieldDefinition(name="account_holder", is_mutable=True, description="Name of the account holder"),
            FieldDefinition(name="client_number", is_mutable=True, description="Client identification number"),
            FieldDefinition(name="date_of_birth", is_mutable=True, description="Customer's date of birth"),
            FieldDefinition(name="customer_account_number", is_mutable=True, description="Customer's account number"),
            FieldDefinition(name="customer_iban", is_mutable=True, description="Customer's IBAN"),
            FieldDefinition(name="customer_bank_name", is_mutable=True, description="Name of the customer's bank"),
            FieldDefinition(name="statement_period", is_mutable=True, description="Statement date range"),
            FieldDefinition(name="statement_date", is_mutable=True, description="Date the statement was created"),
            FieldDefinition(name="opening_balance_debit", is_mutable=True, description="Opening balance debit amount"),
            FieldDefinition(name="opening_balance_credit", is_mutable=True, description="Opening balance credit amount"),
            FieldDefinition(name="opening_balance", is_mutable=True, description="Opening balance total"),
            FieldDefinition(name="transactions", is_mutable=True, description="List of transaction details"),
            FieldDefinition(name="total_debit", is_mutable=True, description="Total debit amount"),
            FieldDefinition(name="total_credit", is_mutable=True, description="Total credit amount"),
            FieldDefinition(name="total", is_mutable=True, description="Total balance"),
            FieldDefinition(name="bank_name", is_mutable=False, description=f"Name of the bank ({bank.capitalize()})"),
            FieldDefinition(name="bank_address", is_mutable=False, description="Bank address"),
            FieldDefinition(name="table_headers", is_mutable=False, description="Headers for transaction table"),
            FieldDefinition(name="customer_service", is_mutable=False, description="Customer service contact information"),
            FieldDefinition(name="footnotes", is_mutable=False, description="Footnotes and disclosures")
        ] if bank == "citibank" else [
            FieldDefinition(name="account_holder", is_mutable=True, description="Name of the account holder"),
            FieldDefinition(name="account_holder_address", is_mutable=True, description="Address of the account holder"),
            FieldDefinition(name="account_number", is_mutable=True, description="Account number"),
            FieldDefinition(name="statement_period", is_mutable=True, description="Statement date range"),
            FieldDefinition(name="summary", is_mutable=True, description="Checking summary data (beginning balance, counts, totals)"),
            FieldDefinition(name="deposits", is_mutable=True, description="List of deposit transactions"),
            FieldDefinition(name="withdrawals", is_mutable=True, description="List of withdrawal transactions"),
            FieldDefinition(name="daily_balances", is_mutable=True, description="Daily ending balances"),
            FieldDefinition(name="logo_path", is_mutable=True, description="Path to the bank logo"),
            FieldDefinition(name="bank_name", is_mutable=False, description=f"Name of the bank ({bank.capitalize()})"),
            FieldDefinition(name="bank_address", is_mutable=False, description="Bank address"),
            FieldDefinition(name="checking_summary_header", is_mutable=False, description="Header for checking summary"),
            FieldDefinition(name="deposits_header", is_mutable=False, description="Header for deposits section"),
            FieldDefinition(name="withdrawals_header", is_mutable=False, description="Header for withdrawals section"),
            FieldDefinition(name="daily_balance_header", is_mutable=False, description="Header for daily balance section"),
            FieldDefinition(name="customer_service", is_mutable=False, description="Customer service contact information"),
            FieldDefinition(name="footnotes", is_mutable=False, description="Footnotes and disclosures")
        ]
        statement_fields = StatementFields(fields=default_fields)
    
    log_path = os.path.join(SYNTHETIC_STAT_DIR, f"template_fields_{bank}.json")
    with open(log_path, 'w', encoding='utf-8') as f:
        json.dump(statement_fields.model_dump(), f, indent=2)
    
    return statement_fields

In [10]:

# Cell 8: Generate populated HTML, PDF, and CSV (Updated template_data)
def generate_populated_html_and_pdf(df: pd.DataFrame, account_holder: str, bank: str, template_dir: str, output_dir: str, account_type: str, template_name: str = None) -> list:
    if bank not in BANK_CONFIG:
        raise ValueError(f"Unsupported bank: {bank}. Supported banks: {list(BANK_CONFIG.keys())}")
    
    templates = [template_name] if template_name else BANK_CONFIG[bank]["templates"]
    for template in templates:
        if template not in BANK_CONFIG[bank]["templates"]:
            raise ValueError(f"Template {template} not supported for {bank}")

    html_dir = os.path.join(output_dir, "HTML")
    pdf_dir = os.path.join(output_dir, "PDF")
    csv_dir = os.path.join(output_dir, "CSV")
    os.makedirs(html_dir, exist_ok=True)
    os.makedirs(pdf_dir, exist_ok=True)
    os.makedirs(csv_dir, exist_ok=True)

    env = Environment(loader=FileSystemLoader(template_dir))

    initial_balance = round(random.uniform(1000, 20000), 2)
    deposits_total = sum(x for x in df['Amount'] if x > 0)
    withdrawals_total = abs(sum(x for x in df['Amount'] if x < 0))
    ending_balance = initial_balance + deposits_total - withdrawals_total
    service_fee = 25 if ending_balance < 5000 else 0
    if service_fee:
        withdrawals_total += service_fee
        ending_balance -= service_fee

    min_date = datetime.strptime(min(df['Date']), "%m/%d").replace(year=2025)
    max_date = datetime.strptime(max(df['Date']), "%m/%d").replace(year=2025)
    statement_date = datetime.now().strftime("%B %d, %Y at %I:%M %p %Z")

    address = fake.address().replace('\n', '<br>')[:100]
    account_holder = account_holder[:50]
    account_number = fake.bban()[:15]

    logo_path = os.path.join(SAMPLE_LOGOS_DIR, BANK_CONFIG[bank]["logo"])
    logo_data = ""
    if os.path.exists(logo_path):
        with open(logo_path, "rb") as img_file:
            logo_data = f"data:image/png;base64,{base64.b64encode(img_file.read()).decode('utf-8')}"

    # Important account information (unchanged from notebook)
    if account_type == "business":
        if bank == "pnc":
            important_info = f"""
            <h3>Important Account Information</h3>
            <p>Effective July 1, 2025, the monthly service fee for {BANK_CONFIG[bank]["account_types"][account_type]} accounts will increase to $15 unless you maintain a minimum daily balance of $5,000, have $2,000 in net purchases on a PNC Business Debit Card, or maintain linked PNC business accounts with a combined balance of $10,000.</p>
            <p>Starting June 30, 2025, PNC will offer enhanced cash flow tools for {BANK_CONFIG[bank]["account_types"][account_type]} accounts via PNC Online Banking, including automated invoice tracking and payment scheduling.</p>
            <p>Effective July 15, 2025, PNC Express Funds fees for {BANK_CONFIG[bank]["account_types"][account_type]} accounts will increase from 2.00% to 2.50% of the check amount over $100; checks between $25 and $100 remain $2.00.</p>
            <p>Effective July 15, 2025, PNC will reduce domestic wire transfer fees to $25 for {BANK_CONFIG[bank]["account_types"][account_type]} accounts, down from $30.</p>
            <p>For questions, visit your local PNC Branch or call the PNC Customer Care Center at <b>1-888-762-2265</b>, available 24/7.</p>
            """
        elif bank == "chase":
            important_info = f"""
            <h3>Important Account Information</h3>
            <p>Effective July 1, 2025, the monthly service fee for {BANK_CONFIG[bank]["account_types"][account_type]} accounts will increase to $20 unless you maintain a minimum daily balance of $2,000, have $2,000 in net purchases on a Chase Business Debit Card, or maintain linked Chase business accounts with a combined balance of $10,000.</p>
            <p>Starting June 30, 2025, Chase will offer enhanced cash flow tools for {BANK_CONFIG[bank]["account_types"][account_type]} accounts via Chase Online, including automated invoice tracking and payment scheduling.</p>
            <p>Effective July 15, 2025, Chase will reduce wire transfer fees to $25 for domestic transfers for {BANK_CONFIG[bank]["account_types"][account_type]} accounts, down from $30.</p>
            <p>For questions, visit your local Chase Branch or call the Chase Customer Care Center at <b>1-800-242-7338</b>, available 24/7.</p>
            """
        elif bank == "citibank":
            important_info = f"""
            <h3>Important Account Information</h3>
            <p>Effective July 1, 2025, the monthly account fee for {BANK_CONFIG[bank]["account_types"][account_type]} accounts will increase to £15 unless you maintain a minimum daily balance of £5,000 or have £2,000 in net purchases on a Citi Business Debit Card per month.</p>
            <p>Starting June 30, 2025, Citibank will offer enhanced cash flow tools for {BANK_CONFIG[bank]["account_types"][account_type]} accounts via Citi Online Banking, including automated invoice tracking and payment scheduling.</p>
            <p>Effective July 15, 2025, Citibank will reduce domestic BACS transfer fees to £20 for {BANK_CONFIG[bank]["account_types"][account_type]} accounts, down from £25.</p>
            <p>For questions, visit citibank.co.uk or contact our Client Contact Centre at 0800 005 555 (or +44 20 7500 5500 from abroad), available 24/7.</p>
            """
        elif bank == "wellsfargo":
            important_info = f"""
            <h3>Important Account Information</h3>
            <p>Effective July 1, 2025, the monthly service fee for {BANK_CONFIG[bank]["account_types"][account_type]} accounts will increase to $20 unless you maintain a minimum daily balance of $5,000, have $2,000 in net purchases on a Wells Fargo Business Debit Card, or maintain linked Wells Fargo business accounts with a combined balance of $10,000.</p>
            <p>Starting June 30, 2025, Wells Fargo will offer enhanced cash flow tools for {BANK_CONFIG[bank]["account_types"][account_type]} accounts via Wells Fargo Online, including automated invoice tracking and payment scheduling.</p>
            <p>Effective July 15, 2025, Wells Fargo will reduce wire transfer fees to $25 for domestic transfers for {BANK_CONFIG[bank]["account_types"][account_type]} accounts, down from $30.</p>
            <p>For questions, visit your local Wells Fargo Branch or call the Wells Fargo Customer Service Center at <b>1-800-869-3557</b>, available 24/7.</p>
            """
    else:  # Personal
        if bank == "pnc":
            important_info = f"""
            <h3>Important Account Information</h3>
            <p>Effective July 1, 2025, the monthly service fee for {BANK_CONFIG[bank]["account_types"][account_type]} accounts will increase to $10 unless you maintain a minimum daily balance of $1,500, have $500 in qualifying direct deposits, or maintain a linked PNC savings account with a balance of $2,500 or more.</p>
            <p>Starting June 30, 2025, PNC will introduce real-time transaction alerts for {BANK_CONFIG[bank]["account_types"][account_type]} accounts via the PNC Mobile app to enhance account monitoring. Enable alerts at pnc.com/alerts.</p>
            <p>Effective July 15, 2025, PNC will waive overdraft fees for transactions of $5 or less and cap daily overdraft fees at two per day for {BANK_CONFIG[bank]["account_types"][account_type]} accounts.</p>
            <p>Between May 1, 2025, and September 30, 2025, PNC will remove the option to print mini statements at PNC ATMs. Use Online Banking, Mobile Banking, or Branch Banking to access account information.</p>
            <p>For questions, visit your local PNC Branch or call the PNC Customer Care Center at <b>1-888-762-2265</b>, available 24/7.</p>
            """
        elif bank == "chase":
            important_info = f"""
            <h3>Important Account Information</h3>
            <p>Effective July 1, 2025, the monthly service fee for {BANK_CONFIG[bank]["account_types"][account_type]} accounts will increase to $15 unless you maintain a minimum daily balance of $1,500, have $500 in qualifying direct deposits, or maintain a linked Chase savings account with a balance of $5,000 or more.</p>
            <p>Starting June 30, 2025, Chase will introduce real-time transaction alerts for {BANK_CONFIG[bank]["account_types"][account_type]} accounts via the Chase Mobile app to enhance account monitoring. Enable alerts at chase.com/alerts.</p>
            <p>Effective July 15, 2025, Chase will waive overdraft fees for transactions of $5 or less and cap daily overdraft fees at two per day for {BANK_CONFIG[bank]["account_types"][account_type]} accounts.</p>
            <p>For questions, visit your local Chase Branch or call the Chase Customer Care Center at <b>1-800-242-7338</b>, available 24/7.</p>
            """
        elif bank == "citibank":
            important_info = f"""
            <h3>Important Account Information</h3>
            <p>Effective July 1, 2025, the monthly account fee for {BANK_CONFIG[bank]["account_types"][account_type]} accounts will increase to £10 unless you maintain a minimum daily balance of £1,500 or have qualifying direct deposits of £500 or more per month.</p>
            <p>Starting June 30, 2025, Citibank will introduce real-time transaction alerts for {BANK_CONFIG[bank]["account_types"][account_type]} accounts via the Citi Mobile UK app. Enable alerts at citibank.co.uk/alerts.</p>
            <p>Effective July 15, 2025, Citibank will waive overdraft fees for transactions of £5 or less and cap daily overdraft fees at two per day for {BANK_CONFIG[bank]["account_types"][account_type]} accounts.</p>
            <p>For questions, visit citibank.co.uk or contact our Client Contact Centre at 0800 005 555 (or +44 20 7500 5500 from abroad), available 24/7.</p>
            """
        elif bank == "wellsfargo":
            important_info = f"""
            <h3>Important Account Information</h3>
            <p>Effective July 1, 2025, the monthly service fee for {BANK_CONFIG[bank]["account_types"][account_type]} accounts will increase to $10 unless you maintain a minimum daily balance of $1,500, have $500 in qualifying direct deposits, or maintain a linked Wells Fargo savings account with a balance of $2,500 or more.</p>
            <p>Starting June 30, 2025, Wells Fargo will introduce real-time transaction alerts for {BANK_CONFIG[bank]["account_types"][account_type]} accounts via the Wells Fargo Mobile app to enhance account monitoring. Enable alerts at wellsfargo.com/alerts.</p>
            <p>Effective July 15, 2025, Wells Fargo will waive overdraft fees for transactions of $5 or less and cap daily overdraft fees at two per day for {BANK_CONFIG[bank]["account_types"][account_type]} accounts.</p>
            <p>For questions, visit your local Wells Fargo Branch or call the Wells Fargo Customer Service Center at <b>1-800-869-3557</b>, available 24/7.</p>
            """

    results = []
    for template_file in templates:
        if bank == "citibank":
            transactions = []
            total_debit = abs(sum(x for x in df['Amount'] if x < 0))
            total_credit = sum(x for x in df['Amount'] if x > 0)
            running_balance = initial_balance
            for _, row in df.sort_values("Date").iterrows():
                amount = row['Amount']
                debit = f"£{abs(amount):,.2f}" if amount < 0 else ""
                credit = f"£{amount:,.2f}" if amount > 0 else ""
                running_balance += amount
                transactions.append({
                    "date": row["Date"],
                    "description": row["Description"],
                    "debit": debit,
                    "credit": credit,
                    "balance": f"£{running_balance:,.2f}",
                    "type": row["Type"]
                })
            template_data = {
                "account_holder": account_holder,
                "client_number": fake.uuid4()[:8],
                "date_of_birth": fake.date_of_birth(minimum_age=18, maximum_age=80).strftime("%m/%d/%Y"),
                "customer_account_number": account_number,
                "customer_iban": f"GB{fake.random_number(digits=2)}CITI{fake.random_number(digits=14)}",
                "customer_bank_name": "Citibank",
                "statement_period": f"{min_date.strftime('%B %d')} through {max_date.strftime('%B %d')}",
                "statement_date": statement_date,
                "opening_balance": f"£{initial_balance:,.2f}",
                "transactions": transactions,
                "total_debit": f"£{total_debit:,.2f}",
                "total_credit": f"£{total_credit:,.2f}",
                "total": f"£{ending_balance:,.2f}",
                "logo_path": logo_data,
                "important_info": important_info,
                "account_type": "Access Checking" if account_type == "personal" else "Business Checking"
            }
        elif bank in ["wellsfargo", "pnc"]:
            transactions = []
            deposits = []
            withdrawals = []
            running_balance = initial_balance
            for _, row in df.sort_values("Date").iterrows():
                amount = row['Amount']
                transaction_type = row['Type']
                deposits_credits = f"${amount:,.2f}" if amount > 0 else ""
                withdrawals_debits = f"${abs(amount):,.2f}" if amount < 0 else ""
                running_balance += amount
                transactions.append({
                    "date": row["Date"],
                    "description": row["Description"],
                    "deposits_credits": deposits_credits,
                    "withdrawals_debits": withdrawals_debits,
                    "ending_balance": f"${running_balance:,.2f}",
                    "type": transaction_type
                })
                if amount > 0:
                    deposits.append({
                        "date": row["Date"],
                        "description": row["Description"],
                        "amount": f"${amount:,.2f}",
                        "type": transaction_type
                    })
                else:
                    withdrawals.append({
                        "date": row["Date"],
                        "description": row["Description"],
                        "amount": f"${abs(amount):,.2f}",
                        "type": transaction_type
                    })
            if service_fee:
                withdrawals.append({
                    "date": max_date.strftime("%m/%d"),
                    "description": "Monthly Service Fee",
                    "amount": f"${service_fee:,.2f}",
                    "type": "other"
                })
                running_balance -= service_fee
            daily_balances = [
                {"date": row["Date"], "amount": f"${row['Balance']:,.2f}"}
                for _, row in df.drop_duplicates(subset="Date").iterrows()
            ]
            summary = {
                "beginning_balance": f"${initial_balance:,.2f}",
                "deposits_total": f"${deposits_total:,.2f}",
                "withdrawals_total": f"${withdrawals_total + (service_fee if service_fee else 0):,.2f}",
                "ending_balance": f"${running_balance:,.2f}",
                "deposits_count": len(deposits),
                "withdrawals_count": len(withdrawals),
                "transactions_count": len(df) + (1 if service_fee else 0),
                "average_balance": f"${round((initial_balance + running_balance) / 2, 2):,.2f}",
                "fees": f"${service_fee:,.2f}",
                "checks_written": sum(1 for w in withdrawals if w["type"] == "check"),
                "pos_transactions": random.randint(0, 10),
                "pos_pin_transactions": random.randint(0, 5),
                "total_atm_transactions": random.randint(0, 8),
                "pnc_atm_transactions": random.randint(0, 5) if bank == "pnc" else 0,
                "other_atm_transactions": random.randint(0, 3),
                "apy_earned": f"{random.uniform(0.01, 0.5):.2f}%" if bank in ["wellsfargo", "pnc"] else "0.00%",
                "days_in_period": (max_date - min_date).days + 1,
                "average_collected_balance": f"${round(random.uniform(initial_balance, running_balance), 2):,.2f}",
                "interest_paid_period": f"${random.uniform(0.1, 10):,.2f}" if bank in ["wellsfargo", "pnc"] else "$0.00",
                "interest_paid_ytd": f"${random.uniform(1, 50):,.2f}" if bank in ["wellsfargo", "pnc"] else "$0.00",
                "overdraft_protection1": f"{bank.capitalize()} Savings Account XXXX1234" if random.choice([True, False]) else "",
                "overdraft_protection2": f"{bank.capitalize()} Credit Line XXXX5678" if random.choice([True, False]) else "",
                "overdraft_status": "Opted-In" if random.choice([True, False]) else "Opted-Out"
            }
            template_data = {
                "account_holder": account_holder,
                "account_holder_address": address,
                "account_number": account_number,
                "statement_period": f"{min_date.strftime('%B %d')}, 2025 – {max_date.strftime('%B %d')}, 2025",
                "statement_date": statement_date,
                "logo_path": logo_data,
                "important_info": important_info,
                "summary": summary,
                "transactions": transactions,
                "deposits": deposits,
                "withdrawals": withdrawals,
                "daily_balances": daily_balances,
                "account_type": "Standard Checking" if account_type == "personal" else "Business Checking"
            }
        else:  # Chase
            deposits = []
            withdrawals = []
            running_balance = initial_balance
            for _, row in df.sort_values("Date").iterrows():
                amount = row['Amount']
                transaction_type = row['Type']
                if amount > 0:
                    deposits.append({
                        "date": row["Date"],
                        "description": row["Description"],
                        "amount": f"${amount:,.2f}",
                        "type": transaction_type
                    })
                else:
                    withdrawals.append({
                        "date": row["Date"],
                        "description": row["Description"],
                        "amount": f"${abs(amount):,.2f}",
                        "type": transaction_type
                    })
                running_balance += amount
            if service_fee:
                withdrawals.append({
                    "date": max_date.strftime("%m/%d"),
                    "description": "Monthly Service Fee",
                    "amount": f"${service_fee:,.2f}",
                    "type": "other"
                })
                running_balance -= service_fee
            statement_start = min_date
            statement_end = max_date
            day_delta = timedelta(days=1)
            balance_map = {}
            current_date = statement_start
            df_sorted = df.sort_values("Date")
            while current_date <= statement_end:
                iso_date = current_date.isoformat()
                daily_transactions = df_sorted[df_sorted["Date"] == current_date.strftime("%m/%d")]
                if not daily_transactions.empty:
                    daily_amount = daily_transactions["Amount"].sum()
                    running_balance += daily_amount
                balance_map[iso_date] = f"${running_balance:,.2f}"
                current_date += day_delta
            daily_balances = [
                {"date": row["Date"], "amount": f"${row['Balance']:,.2f}"}
                for _, row in df_sorted.drop_duplicates(subset="Date").iterrows()
            ]
            summary = {
                "beginning_balance": f"${initial_balance:,.2f}",
                "deposits_count": len(deposits),
                "deposits_total": f"${deposits_total:,.2f}",
                "withdrawals_count": len(withdrawals),
                "withdrawals_total": f"${withdrawals_total + (service_fee if service_fee else 0):,.2f}",
                "ending_balance": f"${running_balance:,.2f}",
                "average_balance": f"${round((initial_balance + running_balance) / 2, 2):,.2f}",
                "fees": f"${service_fee:,.2f}",
                "checks_written": sum(1 for w in withdrawals if w["type"] == "check"),
                "pos_transactions": random.randint(0, 10),
                "pos_pin_transactions": random.randint(0, 5),
                "total_atm_transactions": random.randint(0, 8),
                "pnc_atm_transactions": 0,
                "other_atm_transactions": random.randint(0, 3),
                "apy_earned": "0.00%",
                "days_in_period": (max_date - min_date).days + 1,
                "average_collected_balance": f"${round(random.uniform(initial_balance, running_balance), 2):,.2f}",
                "interest_paid_period": "$0.00",
                "interest_paid_ytd": "$0.00",
                "overdraft_protection1": "Chase Savings Account XXXX1234" if random.choice([True, False]) else "",
                "overdraft_protection2": "Chase Credit Line XXXX5678" if random.choice([True, False]) else "",
                "overdraft_status": "Opted-In" if random.choice([True, False]) else "Opted-Out"
            }
            template_data = {
                "account_holder": account_holder,
                "account_holder_address": address,
                "account_number": account_number,
                "statement_period": f"{min_date.strftime('%B %d')} through {max_date.strftime('%B %d')}",
                "statement_date": statement_date,
                "logo_path": logo_data,
                "important_info": important_info,
                "summary": summary,
                "deposits": deposits,
                "withdrawals": withdrawals,
                "daily_balances": daily_balances,
                "show_fee_waiver": service_fee == 0,
                "account_type": "Total Checking" if account_type == "personal" else "Business Complete Checking",
                "statement_start": statement_start,
                "statement_end": statement_end,
                "day_delta": day_delta,
                "balance_map": balance_map
            }

        template = env.get_template(template_file)
        template_name = os.path.splitext(template_file)[0]
        safe_account_holder = ''.join(c for c in account_holder if c.isalnum() or c in (' ', '_')).replace(' ', '_')
        safe_account_name = (BANK_CONFIG[bank]["account_types"][account_type]).replace(' ', '_')
        html_filename = os.path.join(html_dir, f"bank_statement_{safe_account_name}_{safe_account_holder}_{bank}_{template_name}.html")
        pdf_filename = os.path.join(pdf_dir, f"bank_statement_{safe_account_name}_{safe_account_holder}_{bank}_{template_name}.pdf")
        csv_filename = os.path.join(csv_dir, f"bank_statement_{safe_account_name}_{safe_account_holder}_{bank}.csv")

        rendered_html = template.render(**template_data)

        with open(html_filename, 'w', encoding='utf-8') as f:
            f.write(rendered_html)

        wkhtmltopdf_path = "C:\\Program Files\\wkhtmltopdf\\bin\\wkhtmltopdf.exe"
        config = pdfkit.configuration(wkhtmltopdf=wkhtmltopdf_path)
        options = {
            "enable-local-file-access": "",
            "page-size": "Letter",
            "margin-top": "0.8in",
            "margin-right": "0.9in",
            "margin-bottom": "0.8in",
            "margin-left": "0.9in",
            "encoding": "UTF-8",
            "disable-javascript": "",
            "image-dpi": "300",
            "enable-forms": "",
            "no-outline": "",
            "print-media-type": ""
        }
        try:
            pdfkit.from_string(rendered_html, pdf_filename, configuration=config, options=options)
        except OSError as e:
            print(f"PDF generation failed for {bank} template {template_file}: {e}. Ensure wkhtmltopdf is installed and accessible.")

        df.to_csv(csv_filename, index=False, encoding='utf-8')

        results.append((html_filename, pdf_filename, csv_filename))

    return results


In [ ]:
# Cell 9: Main
if __name__ == "__main__":
    import random
    
    num_transactions = random.randint(3, 12)
    account_holder = fake.company().upper() if random.choice([True, False]) else fake.name().upper()
    template_dir = TEMPLATES_DIR
    output_dir = SYNTHETIC_STAT_DIR
    banks = ["chase", "citibank", "wellsfargo", "pnc"]
    account_types = ["business", "personal"]
    
    for account_type in account_types:
        for bank in banks:
            specific_account_name = BANK_CONFIG[bank]["account_types"][account_type]
            print(f"\nGenerating {specific_account_name} statement for {bank.capitalize()}")
            df = generate_bank_statement(num_transactions, account_holder, account_type)
            
            statement_fields = identify_template_fields(bank, template_dir)
            print(f"Identified Template Fields for {bank.capitalize()}:")
            for field in statement_fields.fields:
                print(f"- {field.name}: {'Mutable' if field.is_mutable else 'Immutable'}, {field.description}")
            
            results = generate_populated_html_and_pdf(df, account_holder, bank, template_dir, output_dir, account_type)
            for html_file, pdf_file, csv_file in results:
                print(f"HTML saved as: {html_file}")
                print(f"PDF saved as: {pdf_file}")
                print(f"CSV saved as: {csv_file}")


Generating Chase Business Complete Checking statement for Chase
Identified Template Fields for Chase:
- account_type: Mutable, Type of the account (e.g., Checking, Savings)
- logo_path: Immutable, URL path to Chase bank logo
- statement_period: Mutable, Period covered by the statement (e.g., Monthly, Quarterly)
- account_number: Mutable, Unique identifier for the account at Chase bank
- account_holder: Mutable, Name of the account holder
- account_holder_address: Mutable, Address of the account holder
- summary.beginning_balance: Immutable, Initial balance at the start of the statement period
- summary.deposits_count: Immutable, Number of deposits made during the statement period
- summary.deposits_total: Immutable, Total amount of deposits made during the statement period
- summary.withdrawals_count: Immutable, Number of withdrawals made during the statement period
- summary.withdrawals_total: Immutable, Total amount of withdrawals made during the statement period
- summary.transacti